In [1]:
import pandas as pd
import plotly.express as px
import numpy as np
import os
import re

In [ ]:
directory ="C:\\Users\\20203666\\Documents\\JBG050-data\\Data"
folders = os.listdir(directory)
folders = folders[1:]
files = []
dic_bug={}
dic_all = {}
for item in folders:
    folder = os.listdir(directory+'\\'+item)
    for file in folder:
        if file.endswith(police_force):
            files.append(directory+'\\'+item+"\\"+file)

df_all = pd.read_csv(files[0])
df_all = df_all[df_all['Crime type']=='Burglary']
i=1
for file in files[1:]:
    df = pd.read_csv(file)
    df = df[df['Crime type']=='Burglary']
    df_all = df_all.append(df, ignore_index=True)
    i=i+1
    print(i)

In [ ]:
dfi=df_all.loc[:,~df_all.columns.isin(['Last outcome category', 'Context', "Crime ID"])].copy()
dfi.dropna()
dfi.rename(columns = {'Month':'Date'}, inplace=True)

In [ ]:
years = []
months = []
for date in dfi['Date']:
    dates = re.findall('([0-9]*)', date)
    year = int(dates[0])-2000
    try:
        month = int(dates[2])
    except:
        print(date)
        month = int(dates[2][1])
    years.append(year)
    months.append(month)
dfi['month'] = months
dfi['year'] = years
df2 = dfi.dropna()
df2.reset_index(inplace=True, drop=True)

In [ ]:
lst_city = []
for lsoa in df2['LSOA name']:

    city = re.findall('[A-Za-z\s]*', lsoa)
    city = city[0][:-1]
    lst_city.append(city)
df2['city'] = lst_city

In [ ]:
df_price = pd.read_csv("C:\\Users\\20203666\\Documents\\JBG050-data\\house_price.csv")
df_price

In [ ]:
for i in range(len(df_price)):
    print(i)
    price = df_price.loc[i,'house price']
    lsoa = df_price.loc[i, 'LSOA code']
    df2.loc[df2['LSOA code']==lsoa, 'median house price lsoa'] = price

In [ ]:
df = df2.copy()

In [ ]:
group_city =df.groupby(['Date', 'city']).count()
lst_city = []
c=0
for i in range(len(df)):
    if c==10000:
        print(i)
        c=0
    c+=1
    bur = group_city.loc[(df.loc[i, 'Date'],df.loc[i, 'city']),'Unnamed: 0']
    lst_city.append(bur)
df['burglarys city'] = lst_city

In [ ]:
group_lsoa =df.groupby(['Date', 'LSOA code']).count()
lst_lsoa = []
c=0
for i in range(len(df)):
    if c==10000:
        print(i)
        c=0
    c+=1
    bur = group_lsoa.loc[(df.loc[i, 'Date'],df.loc[i, 'LSOA code']),'Unnamed: 0']
    lst_lsoa.append(bur)
df['burglarys lsoa'] = lst_lsoa

In [ ]:
df = df[df['median house price lsoa']!=0]
df.reset_index(inplace=True, drop=True)
df.to_csv("C:\\Users\\20203666\\Documents\\JBG050-data\\final_burglary_data_lsoa.csv")

In [ ]:
df1 = df.groupby(['Date', 'city']).median()
df1['city'] = [df1.index[i][1] for i in range(len(df1.index))]
df1 = df1.droplevel(0)
df1.reset_index(inplace=True, drop=True)
df1 = df1.rename(columns = {'median house price lsoa': 'median house price'})
df1

In [ ]:
lst_city = list(df['city'].unique())
dict_city = {}
for i in range(len(df)):
    #print(i)
    dict_city[df.loc[i, 'city']] = df.loc[i, 'Falls within']

In [ ]:
df['police district'] = 0
for i in range(len(df1)):
    district = dict_city[df1.loc[i, 'city']]
    df1.loc[i, 'police district'] = district

In [ ]:
df1.to_csv('C:\\Users\\20203666\\Documents\\JBG050-data\\final_burglary_data_city.csv')